In [1]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [2]:
def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

Start with The encoder decoder. The Encoder will firt encode input of of symbols to a contignous representation. The decoder The generates an output sequence of symbols dependent on the previously generated symbols, i.e auto-regressive.

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, source_embeddings, target_embeddings, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.source_embeddings = source_embeddings
        self.target_embeddings = target_embeddings
        self.generator = generator

    def forward(self, source, target, source_mask, target_mask):

        # In the forward step we will first encode the source symbols, 
        # then generate new symbols using the decoder

        encoded = self.encode(source, source_mask)
        decoded = self.decode(encoded, source_mask, target, target_mask)

        return decoded
    
    def encode(self, source, source_mask):
        # Generate embeddings for source and encode
        src_embedding = self.source_embedding(source)
        encoded = self.encoder(src_embedding, source_mask)

        return encoded
    
    def decode(self, memory, source_mask, target, target_mask):

        # Generate embeddings for target, and decode with embeddings and memory
        tgt_embedding = self.target_embedding(target)
        decoded = self.decoder(tgt_embedding, memory, source_mask, target_mask)

        return decoded




In [3]:
class Generator(nn.Module):
    # Linnear softmax step func

    def __init__(self, d_model, vocab):

        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

        def forward(self, x):
            return log_softmax(self.proj(x), dim=-1)

Encoder stack will consist of N = 6 layers

In [4]:
def clones(module, N):

    # generating a list of 6 independent modules to train on. 
    # Using deepcopy ensures the parameters are independetn when training
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [5]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):

        for layer in self.layers:
            x = layer(x, mask)

        return self.norm(x)

In [6]:
class LayerNorm(nn.Module):

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)

        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

SyntaxError: invalid syntax (2534558847.py, line 4)